In [1]:
import music21

In [2]:
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"

In [4]:
music21.converter.parse(little_star).show('mid')

#### 계이름과 숫자를  상호 변환하는 표 - dic

In [9]:
temp1 = [1,2,3,4,5,6,7]
temp2 = ['c','d','e','f','g','a','b']
note2num = {}
for key,value in zip(temp2,temp1):
    note2num[key] = value
num2note = {}
for key,value in zip(temp1,temp2):
    num2note[key] = value  
    

In [10]:
note2num

{'c': 1, 'd': 2, 'e': 3, 'f': 4, 'g': 5, 'a': 6, 'b': 7}

In [11]:
num2note

{1: 'c', 2: 'd', 3: 'e', 4: 'f', 5: 'g', 6: 'a', 7: 'b'}

#### ABC 표기를 시계열 데이터로 변환

In [22]:
def abc2timeseries(s):
    notes = s.split(' ')[2:]
    seq = []
    for i in notes:
        seq.append([note2num[i[0]], int(i[1])])
    return seq

#### 시계열 데이터를 ABC 표기로 변환

In [12]:
def timeseries2abc(t):
    s = 'tinynotation: 4/4'
    for i in t:
        s = s+' '+num2note[i[0]]+str(i[1])
    return s

#### 원핫 코드로 변환하는 표

In [15]:
# 2음절,4음절,8음절
onehot = [[i,2] for i in range(1,8)] +[[i,4] for i in range(1,8)]+[[i,8] for i in range(1,8)]

#### 레이블을 원핫 코드로 변환

In [64]:
import numpy as np
def to_onehot(la):
    t = []
    for i in range(len(la)):
        a = np.zeros(len(onehot))
        a[onehot.index(list(la[i]))] = 1.0
        t.append(a)
    return np.array(t)

#### 시계열 데이터를 훈련 집합으로 자름

In [65]:
def seq2dataset(seq,window,horizon):
    X,Y = [],[]
    for i in range(len(seq) - (window+horizon)+1):
        x = seq[i:(i+window)]
        y = (seq[i+window+horizon-1])
        X.append(x); Y.append(y)
    return np.array(X,dtype=object), np.array(Y,dtype=object) # 경고메세지에따라서 dtype=object 추가

In [66]:
# 윈도우크기, 수평선 계수
w=8; h=1

In [67]:
little_star

'tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2'

In [68]:
seq = abc2timeseries( little_star)
X,Y = seq2dataset(seq,w,h)
print(X.shape, Y.shape)
print(X[0], Y[0])

(34, 8, 2) (34, 2)
[[1 4]
 [1 4]
 [5 4]
 [5 4]
 [6 4]
 [6 4]
 [5 2]
 [4 4]] [4 4]


#### 훈련집합 구축

In [69]:
split = int(len(X)*1.0)  # 100%를 훈련 집합으로 사용
x_train = X[:split]; y_train = Y[:split]
y_train = to_onehot(y_train)


#### LSTM 모델 설계와 학습

In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow as  tf

In [83]:
model = Sequential()
model.add(LSTM(units=128, activation='relu', input_shape=x_train[0].shape))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer='Adam',metrics=['accuracy'])
model.fit(x_train.astype(np.float32),y_train.astype(np.float32),epochs=200,batch_size=1, verbose=2)

Epoch 1/200
34/34 - 1s - loss: 2.5809 - accuracy: 0.1765 - 896ms/epoch - 26ms/step
Epoch 2/200
34/34 - 0s - loss: 2.2092 - accuracy: 0.2059 - 70ms/epoch - 2ms/step
Epoch 3/200
34/34 - 0s - loss: 2.0449 - accuracy: 0.2059 - 84ms/epoch - 2ms/step
Epoch 4/200
34/34 - 0s - loss: 1.8823 - accuracy: 0.2647 - 77ms/epoch - 2ms/step
Epoch 5/200
34/34 - 0s - loss: 1.7716 - accuracy: 0.3235 - 70ms/epoch - 2ms/step
Epoch 6/200
34/34 - 0s - loss: 1.5632 - accuracy: 0.4118 - 80ms/epoch - 2ms/step
Epoch 7/200
34/34 - 0s - loss: 1.6570 - accuracy: 0.4412 - 85ms/epoch - 3ms/step
Epoch 8/200
34/34 - 0s - loss: 1.4523 - accuracy: 0.5294 - 76ms/epoch - 2ms/step
Epoch 9/200
34/34 - 0s - loss: 1.5017 - accuracy: 0.5588 - 80ms/epoch - 2ms/step
Epoch 10/200
34/34 - 0s - loss: 1.3921 - accuracy: 0.4706 - 92ms/epoch - 3ms/step
Epoch 11/200
34/34 - 0s - loss: 1.4224 - accuracy: 0.3529 - 99ms/epoch - 3ms/step
Epoch 12/200
34/34 - 0s - loss: 1.2358 - accuracy: 0.5588 - 100ms/epoch - 3ms/step
Epoch 13/200
34/34 - 0

Epoch 101/200
34/34 - 0s - loss: 0.0664 - accuracy: 0.9706 - 73ms/epoch - 2ms/step
Epoch 102/200
34/34 - 0s - loss: 0.0589 - accuracy: 0.9706 - 84ms/epoch - 2ms/step
Epoch 103/200
34/34 - 0s - loss: 0.0637 - accuracy: 0.9412 - 77ms/epoch - 2ms/step
Epoch 104/200
34/34 - 0s - loss: 0.0649 - accuracy: 0.9706 - 84ms/epoch - 2ms/step
Epoch 105/200
34/34 - 0s - loss: 0.0583 - accuracy: 0.9706 - 62ms/epoch - 2ms/step
Epoch 106/200
34/34 - 0s - loss: 0.0522 - accuracy: 0.9706 - 72ms/epoch - 2ms/step
Epoch 107/200
34/34 - 0s - loss: 0.0639 - accuracy: 0.9412 - 79ms/epoch - 2ms/step
Epoch 108/200
34/34 - 0s - loss: 0.0524 - accuracy: 0.9706 - 100ms/epoch - 3ms/step
Epoch 109/200
34/34 - 0s - loss: 0.0568 - accuracy: 0.9412 - 92ms/epoch - 3ms/step
Epoch 110/200
34/34 - 0s - loss: 0.0609 - accuracy: 0.9412 - 87ms/epoch - 3ms/step
Epoch 111/200
34/34 - 0s - loss: 0.0627 - accuracy: 0.9412 - 73ms/epoch - 2ms/step
Epoch 112/200
34/34 - 0s - loss: 0.0322 - accuracy: 1.0000 - 72ms/epoch - 2ms/step
Epo

Epoch 199/200
34/34 - 0s - loss: 5.3409e-05 - accuracy: 1.0000 - 74ms/epoch - 2ms/step
Epoch 200/200
34/34 - 0s - loss: 5.2569e-05 - accuracy: 1.0000 - 72ms/epoch - 2ms/step


In [82]:
x_train.astype(np.float32)

array([[[1., 4.],
        [1., 4.],
        [5., 4.],
        [5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.]],

       [[1., 4.],
        [5., 4.],
        [5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.]],

       [[5., 4.],
        [5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.]],

       [[5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.]],

       [[6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.],
        [2., 4.]],

       [[6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.],
        [2., 4.],
        [2., 4.]],

       [[5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.],
        [2., 4.],
        [2., 4.]